## 🧩 Setup and Installation

### 📦 Required Packages

This notebook uses just a few basic packages:
* `hypha-rpc` - To connect to the Hypha server and access the Tabula Trainer application
* `aiortc` - To handle WebRTC connections for peer-to-peer communication
* `numpy` - To handle array data for images and model outputs

The minimal requirements allow this notebook to run even in a Pyodide environment directly in your browser.

In [1]:
try:
    # For pyodide in the browser
    import micropip

    await micropip.install(["pyodide-http", "aiortc", "hypha-rpc", "numpy"])

    # 2. Patch requests
    import pyodide_http

    pyodide_http.patch_all()  # Patch all libraries
except ImportError:
    # For native python with pip
    import subprocess

    subprocess.call(["pip", "install", "aiortc", "hypha-rpc", "numpy"])

import asyncio

from hypha_rpc import connect_to_server, get_rtc_service, login

## 🔌☁️ Connect to the Hypha Server

Let's connect to the Hypha server, which lets us communicate with the BioEngine infrastructure.

By default, we'll use the public server at `https://hypha.aicell.io`. If you're using your own server, change the `SERVER_URL` variable below.

In [2]:
SERVER_URL = "https://hypha.aicell.io"

In [3]:
token = await login({"server_url": SERVER_URL})
server = await connect_to_server(
    {"server_url": SERVER_URL, "token": token, "method_timeout": 3000}
)

Please open your browser and login at https://hypha.aicell.io/public/apps/hypha-login/?key=qfYMjN4yyUGDWGuFkYknFS


In [4]:
services = await server.list_services({"type": "bioengine-worker"})

for i, service in enumerate(services):
    if i == 0:
        print("✅ BioEngine worker services available in your workspace:\n")
    if i == len(services) - 1:
        print(" * ", end="")
        PRIVATE_BIOENGINE = service.id
    else:
        print("   ", end="")
    print(f"BioEngine worker service ID: {service.id}")

if len(services) == 0:
    print(
        f"⚠️  No BioEngine worker service available in workspace '{server.config.workspace}'."
    )
else:
    bioengine_worker = await server.get_service(PRIVATE_BIOENGINE)

✅ BioEngine worker services available in your workspace:

 * BioEngine worker service ID: ws-user-github|49943582/T5TQN7kCyupiTJwfgeTruM:bioengine-worker


In [ ]:
app_id = "tabula-trainer"
status = None
iterations = 0
while status != "RUNNING":
    await asyncio.sleep(1)
    app_status = await bioengine_worker.get_application_status([app_id])
    status = app_status["status"]
    if status == "RUNNING":
        print("✅ Tabula-Trainer is ready to use!", " " * 20)
        service_ids = app_status["service_ids"][0]
        WEBSOCKET_SERVICE_ID = service_ids["websocket_service_id"]
        WEBRTC_SERVICE_ID = service_ids["webrtc_service_id"]
    else:
        waiting_dots = "." * iterations + " " * (5 - iterations)
        print(
            f"⚠️ Tabula-Trainer is currently {status}. Waiting{waiting_dots}",
            end="\r",
        )
        iterations += 1
        if iterations > 5:
            iterations = 0

✅ Tabula-Trainer is ready to use!                     


---

In [6]:
websocket_service = await server.get_service(WEBSOCKET_SERVICE_ID)

In [7]:
datasets = await websocket_service.list_datasets()
datasets

{'thymus': {'name': 'Thymus-RNA', 'description': 'Single-cell RNA-seq datasets of human thymus.', 'id': 'thymus', 'authorized_users': ['nils.mech@gmail.com'], 'files': {'filter_22.zarr': {'description': 'filter_22', 'version': '1.0.0', 'n_samples': 3077, 'n_vars': 1200}}}}

In [8]:
trainer_properties = await websocket_service.get_properties(context=server.config)
print(trainer_properties)

{'client_id': 'c4a632b3-f072-465e-b380-7f288c6ed64b', 'client_name': 'c4a632b3-f072-465e-b380-7f288c6ed64b', 'train_samples': 2769, 'val_samples': 308, 'device': 'cuda:0'}


In [9]:
parameters = await websocket_service.get_parameters(context=server.config)
parameters

[array([[ 0.17139289, -0.17715943, -0.22312236, ..., -0.11520194,
         -0.00508399, -0.21330027],
        [ 0.07763056,  0.12464914, -0.08731866, ..., -0.13979803,
         -0.05799699, -0.11486075],
        [ 0.09508418, -0.05696251, -0.101274  , ..., -0.1936435 ,
          0.05320034, -0.12226827],
        ...,
        [-0.00254937,  0.01048784,  0.00297853, ..., -0.00646115,
          0.00119404, -0.04858388],
        [ 0.03245753,  0.02417042,  0.00660454, ...,  0.00559023,
          0.02646604,  0.01600402],
        [ 0.01080064, -0.00661317,  0.01213786, ..., -0.01016473,
         -0.00119156, -0.00622535]], shape=(576, 192), dtype=float32),
 array([ 4.53854293e-01, -4.06584423e-03,  2.50214607e-01, -4.89416420e-02,
        -3.28138590e-01,  2.44596258e-01,  3.16067964e-01,  9.03842002e-02,
         3.21437597e-01,  5.85248470e-02,  7.78527111e-02,  2.14828104e-01,
        -1.42166018e-01,  7.49390200e-02, -1.95634544e-01, -3.54196429e-01,
         2.87195355e-01, -1.88991904

In [13]:
await websocket_service.start_fit(parameters)

{'status': 'started', 'message': 'Fit task started in background'}

In [21]:
await websocket_service.cancel_fit()

{'status': 'cancelled', 'message': 'Fit task cancellation requested. Note: PyTorch Lightning training may continue until the current epoch completes.'}

In [22]:
fit_result = await websocket_service.get_fit_result()

if fit_result["status"] == "completed":
    new_parameters, train_samples, metrics = fit_result["result"].values()
    print(metrics)
else:
    print(fit_result)

{'status': 'failed', 'error': 'Task was cancelled by user'}


In [ ]:
await websocket_service.start_evaluate(new_parameters)

{'status': 'started', 'message': 'Evaluate task started in background'}

In [79]:
await websocket_service.cancel_evaluate()

{'status': 'cancelled', 'message': 'Evaluate task cancellation requested. Note: PyTorch Lightning evaluation may continue until completion.'}

In [25]:
eval_result = await websocket_service.get_evaluate_result()

if eval_result["status"] == "completed":
    loss, num_samples, metrics = eval_result["result"].values()
    print(metrics)
else:
    print(eval_result)

{'loss': 3.0683701038360596}


---

In [126]:
# Connect via WebRTC for peer-to-peer communication
peer_connection = await get_rtc_service(server, WEBRTC_SERVICE_ID)

webrtc_service = await peer_connection.get_service(app_id)

In [127]:
await webrtc_service.list_datasets(context=server.config)

{'thymus': {'name': 'Thymus-RNA', 'description': 'Single-cell RNA-seq datasets of human thymus.', 'id': 'thymus', 'authorized_users': ['nils.mech@gmail.com'], 'files': {'filter_22.zarr': {'description': 'filter_22', 'version': '1.0.0', 'n_samples': 3077, 'n_vars': 1200}}}}

In [128]:
await webrtc_service.get_properties(context=server.config)

{'client_id': '47597106-93e6-4c7b-882f-bce85be3de3b', 'client_name': '47597106-93e6-4c7b-882f-bce85be3de3b', 'train_samples': 2769, 'val_samples': 308, 'device': 'cuda:0', 'loss': 2.4860804080963135}

In [ ]:
await webrtc_service.get_parameters(context=server.config)